# Advent of Code 2021 - Day 16

In [1]:
EXAMPLE = 0
filename = "input.txt"
if EXAMPLE: filename = "example.txt"
    
data = []
with open(filename, "r") as f:
    for line in f:
        data += [line.strip()]
        
data[0]

'220D4B80491FE6FBDCDA61F23F1D9B763004A7C128012F9DA88CE27B000B30F4804D49CD515380352100763DC5E8EC000844338B10B667A1E60094B7BE8D600ACE774DF39DD364979F67A9AC0D1802B2A41401354F6BF1DC0627B15EC5CCC01694F5BABFC00964E93C95CF080263F0046741A740A76B704300824926693274BE7CC880267D00464852484A5F74520005D65A1EAD2334A700BA4EA41256E4BBBD8DC0999FC3A97286C20164B4FF14A93FD2947494E683E752E49B2737DF7C4080181973496509A5B9A8D37B7C300434016920D9EAEF16AEC0A4AB7DF5B1C01C933B9AAF19E1818027A00A80021F1FA0E43400043E174638572B984B066401D3E802735A4A9ECE371789685AB3E0E800725333EFFBB4B8D131A9F39ED413A1720058F339EE32052D48EC4E5EC3A6006CC2B4BE6FF3F40017A0E4D522226009CA676A7600980021F1921446700042A23C368B713CC015E007324A38DF30BB30533D001200F3E7AC33A00A4F73149558E7B98A4AACC402660803D1EA1045C1006E2CC668EC200F4568A5104802B7D004A53819327531FE607E118803B260F371D02CAEA3486050004EE3006A1E463858600F46D8531E08010987B1BE251002013445345C600B4F67617400D14F61867B39AA38018F8C05E430163C6004980126005B801CC0417080106005000CB4002D7A801AA0062

# Part 1

In [2]:
def add_leading_zeros_to_make_multiple_of_four(in_b):    
    if len(in_b) % 4 != 0:        
        new_len = len(in_b)
        while(True):
            new_len += 1
            if new_len % 4 == 0:
                break
        new = ('0' * (new_len - len(in_b))) + in_b
        return new
    
    else:
        return in_b
    
add_leading_zeros_to_make_multiple_of_four('111000000000000110111101000101001010010001001000000000')

'00111000000000000110111101000101001010010001001000000000'

In [3]:
class Packet_Node:
    def __init__(self):
        self.children = []
        self.version = None
        self.type_id = None
        self.num_bits = None
        self.literal = None
        
    def Print_Tree(self, depth = 0):
        print('\t' * depth, 'Node (Size: {}):'.format(self.num_bits), ', Children ({})'.format(len(self.children)), ', Type ID:', self.type_id, ', Version:', self.version, ', Literal:', self.literal)
        for x in self.children:
            x.Print_Tree(depth + 1)
        
        
Packet_Node().Print_Tree()

 Node (Size: None): , Children (0) , Type ID: None , Version: None , Literal: None


In [4]:
def Grab_Literal(packet):
    
    literal_b = ''
    number = packet[6:]
    bit_count = 0
    for i in range(0, len(number), 5):
        chunk = number[i: i + 5]
        bit_count += 5
        if chunk[0] == '1':
            literal_b += chunk[1:]
        elif chunk[0] == '0':
            literal_b += chunk[1:]
            break
            
    remaining = packet[6 + bit_count: ]
    return literal_b, bit_count, remaining

In [5]:
def Create_Tree_From_Packets(packet, parent_node, num_children_packets = None, depth = 0, debug = False):
    if debug: print('\nCreate_Tree_From_Packets:', packet, ', Read Limit:', num_children_packets, ', Recursion Depth:', depth)

    node = Packet_Node()
    parent_node.children += [node]

    version_b = packet[:3]
    type_id_b = packet[3:6]
    version = int(version_b, 2)
    type_id = int(type_id_b, 2)
    
    node.version = version
    node.type_id = type_id
    
    if debug: print('\tPacket:', version_b, type_id_b, packet[6:])
    if debug: print('\tVersion:', version_b, '=', version)
    if type_id == 4: 
        if debug: print('\tType ID:', type_id_b, '=', type_id, '(Literal)')
    else: 
        if debug: print('\tType ID:', type_id_b, '=', type_id)
        
    # Literal Packet
    if type_id == 4:
        if debug: print('\n\tLiteral Packet!\n')
        literal_b, bit_count, remaining = Grab_Literal(packet)
                
        if debug: print('\tLiteral (Size in bits: {}):'.format(len(literal_b)), literal_b, '=', int(literal_b, 2))
        if debug: print('\tEncoding Bit Count:', bit_count)
        
        if debug: print('\t{} {} {}'.format(version_b, type_id_b, packet[6:]))
        if debug: print('\t{} {} {}'.format('VVV', 'TTT', 'L' * bit_count))
        if debug: print('\tRemaining bits:', remaining)

        if len(remaining) and remaining != len(remaining) * '0':
            if num_children_packets:
                if num_children_packets > 1:
                    if debug: print('\tRecursing from literal with read limit on remaining!')
                    Create_Tree_From_Packets(remaining, parent_node, num_children_packets - 1, depth = depth + 1, debug = debug)
            else:
                if debug: print('\tRecursing from literal on remaining!')
                Create_Tree_From_Packets(remaining, parent_node, depth = depth + 1, debug = debug)
        else:
            if debug: print('\tNo remaining or all zeros!')
            
        node.literal = int(literal_b, 2)
        node.num_bits = 3 + 3 + bit_count
        
    # Not Literal
    else:
        length_type_ID = packet[6]
        if debug: print('\n\tNot Literal!')
        if debug: print('\tLength Type ID:', length_type_ID)
        
        # Bit Count
        if length_type_ID == '0':
            if debug: print('\n\tBit Count!\n')
            total_length_b = packet[7 : 7  + 15]
            total_length = int(total_length_b, 2)
            node.num_bits = 7 + 15 + total_length

            if debug: print('\tTotal length in bits of sub-packets:', total_length_b, "=", total_length)

            children_packets = packet[7 + 15 : 7 + 15 + total_length]
            assert len(children_packets) == total_length
        
        
            if debug: print('\tChildren bits:', children_packets)
            
            if debug: print('\t{} {} {} {} {}'.format(version_b, type_id_b, length_type_ID, total_length_b, packet[7 + 15 :]))
            if debug: print('\t{} {} {} {} {}'.format('VVV', 'TTT', 'I', 'X' * 15, 'C' * len(children_packets)))
            
            if debug: print('\tRecursing from bit count on children!')
            Create_Tree_From_Packets(children_packets, node, depth = depth + 1, debug = debug)
            
            remaining = packet[7 + 15 + total_length: ]            
            if debug: print('\tRemaining bits:', remaining)

            if len(remaining) and remaining != len(remaining) * '0':
                if num_children_packets:
                    if num_children_packets > 1:
                        if debug: print('\tRecursing from bit count with read limit on remaining!')
                        Create_Tree_From_Packets(remaining, parent_node, num_children_packets - 1, depth = depth + 1, debug = debug)
                else:
                    if debug: print('\tRecursing from bit count on remaining!', depth = depth + 1, debug = debug)
                    Create_Tree_From_Packets(remaining, parent_node)
            else:
                if debug: print('\tNo remaining or all zeros!')

        # Packet Count
        else:
            if debug: print('\n\tPacket Count!\n')
            num_packets_b = packet[7 : 7 + 11]
            num_packets = int(num_packets_b, 2)
            
            if debug: print('\tNumber of sub-packets immediately contained:', num_packets_b, "=", num_packets)
            
            all_after = packet[7 + 11 : ]
            if debug: print('\tAll after bits:', all_after)
            
            if debug: print('\t{} {} {} {} {}'.format(version_b, type_id_b, length_type_ID, num_packets_b, packet[7 + 11 :]))
            if debug: print('\t{} {} {} {} {}'.format('VVV', 'TTT', 'I', 'X' * 11, '?' * len(all_after)))
            

            if debug: print('\tRecursing from bit packet count on all after!')
            Create_Tree_From_Packets(all_after, node, num_packets, depth = depth + 1, debug = debug)
            
            if debug: print('\tExpected number of sub-packets:', num_packets, 'Actual Number of sub-packets', len(node.children))
            
            assert len(node.children) == num_packets 

            # At this point we know where it ends
            total_length = sum([x.num_bits for x in node.children])
            if debug: print('my total_length', [x.num_bits for x in node.children], total_length)
            node.num_bits = 7 + 11 + total_length
            
            remaining = packet[7 + 11 + total_length : ]            
            if debug: print('\tRemaining bits:', remaining)

            if len(remaining) and remaining != len(remaining) * '0':
                if num_children_packets:
                    if num_children_packets > 1:
                        if debug: print('\tRecursing from packet count with read limit on remaining!')
                        Create_Tree_From_Packets(remaining, parent_node, num_children_packets - 1, depth = depth + 1, debug = debug)
                else:
                    if debug: print('\tRecursing from packet count on remaining!')
                    Create_Tree_From_Packets(remaining, parent_node, depth = depth + 1, debug = debug)
            else:
                if debug: print('\tNo remaining or all zeros!')
            

            

In [6]:
#in_h = 'D2FE28'
#in_h = '38006F45291200'
#in_h = 'EE00D40C823060'

#in_h = '8A004A801A8002F478'
#in_h = '620080001611562C8802118E34'
#in_h = 'C0015000016115A2E0802F182340'
#in_h = 'A0016C880162017C3686B18A3D4780'

#in_h = 'C200B40A82'
#in_h = '04005AC33890'
#in_h = '880086C3E88112'
#in_h = 'CE00C43D881120'
#in_h = 'D8005AC2A8F0'
#in_h = 'F600BC2D8F'
#in_h = '9C005AC2F8F0'
#in_h = '9C0141080250320F1802104A08'

in_h = data[0]

debug = False

in_b_raw = bin(int(in_h, 16))[2:]
if debug: print('Raw (Size: {}):'.format(len(in_b_raw)), in_b_raw)

packet = add_leading_zeros_to_make_multiple_of_four(in_b_raw)
if debug: print('Padded (Size: {}):'.format(len(packet)), packet)

root = Packet_Node()
Create_Tree_From_Packets(packet, root, None, debug = False)
root.Print_Tree()

 Node (Size: None): , Children (1) , Type ID: None , Version: None , Literal: None
	 Node (Size: 5476): , Children (53) , Type ID: 0 , Version: 1 , Literal: None
		 Node (Size: 107): , Children (4) , Type ID: 3 , Version: 1 , Literal: None
			 Node (Size: 11): , Children (0) , Type ID: 4 , Version: 4 , Literal: 15
			 Node (Size: 41): , Children (0) , Type ID: 4 , Version: 7 , Literal: 199071281
			 Node (Size: 21): , Children (0) , Type ID: 4 , Version: 7 , Literal: 499
			 Node (Size: 16): , Children (0) , Type ID: 4 , Version: 5 , Literal: 190
		 Node (Size: 93): , Children (2) , Type ID: 1 , Version: 6 , Literal: None
			 Node (Size: 16): , Children (0) , Type ID: 4 , Version: 2 , Literal: 240
			 Node (Size: 59): , Children (2) , Type ID: 5 , Version: 4 , Literal: None
				 Node (Size: 21): , Children (0) , Type ID: 4 , Version: 7 , Literal: 3410
				 Node (Size: 16): , Children (0) , Type ID: 4 , Version: 1 , Literal: 201
		 Node (Size: 33): , Children (1) , Type ID: 3 , Version:

In [7]:
def Sum_Versions(root):
    if len(root.children):
        return root.version + sum([Sum_Versions(x) for x in root.children])
    else:
        return root.version
            
Sum_Versions(root.children[0])

977

In [8]:
def Compute_From_Tree(root):
            
    if root.type_id == 4: # Literal
        return root.literal
    
    if root.type_id == 0: # Sum
        res = [Compute_From_Tree(x) for x in root.children]
        return sum(res)
        
    if root.type_id == 1: # Prod
        res = 1
        for x in root.children:
            res *= Compute_From_Tree(x)
        return res

    if root.type_id == 2: # Min
        return min([Compute_From_Tree(x) for x in root.children])
        
    if root.type_id == 3: # Max
        return max([Compute_From_Tree(x) for x in root.children])
    
    if root.type_id == 5: # GE
        res = [Compute_From_Tree(x) for x in root.children]
        return int(res[0] > res[1])
    
    if root.type_id == 6: # LE
        res = [Compute_From_Tree(x) for x in root.children]
        return int(res[0] < res[1])
    
    if root.type_id == 7: # Equal
        res = [Compute_From_Tree(x) for x in root.children]
        return int(res[0] == res[1])

Compute_From_Tree(root.children[0])

101501020883